In [1]:
from textblob import TextBlob
from generatengrams import ngrammatch
#from autocorrect import spell
from Contexts import *
import json
from Intents import *
import emoji
import random
import os
import re

In [2]:
import json
# Read Json File
def getTickets(From, Tos, Dates,Connection):
    with open('tickets.json') as ticketfile:
        #TicketList = json.load(ticketfile)
        try:
            TicketList = json.load(ticketfile)
        except:
            print("no json file loaded for finding Flights")

    #print(ticketList)
        #print("given attributes...:",From, Tos, Dates,Connection)
    
        FlightsFound = [str(i+1) + ") " + ticket["Airlines"]  for i, ticket in enumerate(filter(lambda ticket: ticket["From"] == From and (ticket["Tos"]) == Tos and (ticket["Dates"] == Dates)and (ticket["Connection"] == Connection) , TicketList))]
        Ref_no_flight = [str(i+1) + ") " + ticket["Ref_no"]  for i, ticket in enumerate(filter(lambda ticket: ticket["From"] == From and (ticket["Tos"]) == Tos and (ticket["Dates"] == Dates)and (ticket["Connection"] == Connection) , TicketList))]
    return FlightsFound ,Ref_no_flight

In [3]:
import json

# Read Json File

def check_actions(current_intent, attributes, context):
    '''This function performs the action for the intent
    as mentioned in the intent config file'''
    '''Performs actions pertaining to current intent
    for action in current_intent.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()
    '''
    if current_intent.action == 'Get_Tickets':
        
        #print("Some Flights matching for You... \n\n")
        
        Tos = attributes['Tos']
        From = attributes['From']
        Dates = attributes['Dates']
        Connection = attributes['Connection']
        
               
        '''get all flights list from the database'''
        FlightsFound,Ref_no_flight = getTickets(From, Tos, Dates,Connection) 
                
        if len(FlightsFound) == 0:
            print("No FlightsFound Found")
        else:
            print("Some Flights matching for You... => \n\n{0}".format("\n".join(FlightsFound)))
            print("Please shoose the flight for booking")
            BT=input()
            #print(FlightsFound[(int(BT)-1)])
            
            print("\n Congratulations **** Your Flight:"+FlightsFound[(int(BT)-1)]+" Booked succesfully. \n Referance NO. for your booking : => ",(Ref_no_flight[(int(BT)-1)]))
            
    #if current_intent.action == 'Hello_Greeting':
    #    print("hello.......")
    #    context.active = False           
    context = IntentComplete()
    return 'action: ' + current_intent.action, context


def check_required_params(current_intent, attributes, context):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.required:
            if para.name not in attributes:
                #Example of where the context is born, implemented in Contexts.py
                #if para.name=='RegNo':
                    #context = GetRegNo()
                #if para.name=='cuisine':
                    #context = cuisine()
                #if para.name=='size':
                    #context = size()
                #if para.name=='priceranges':
                   # context = priceranges()
                #if para.name=='costpreference':
                    #context = costpreference()
                #returning a random prompt frmo available choices.
                return random.choice(para.prompts), context

    return None, context


def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    #print(user_input)
    uinput = user_input
    if uinput == "cheap":
        uinput = "True"
    if uinput == "fast":
        uinput = "False"
    #uinput = TextBlob(user_input).correct().string
    #uinput = TextBlob(user_input).translate(to='en')
    #print(uinput)
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(uinput, context, attributes)
    #print(cleaned_input,"\n",attributes)
    return attributes, uinput

def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'], intent['actions'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    
    #Scoring Algorithm, can be changed.
    scores = ngrammatch(clean_input)
    
    #choosing here the intent with the highest score
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    # print clean_input
    #print 'scores', scores
    
          
    #def getAction(clean_input):
    if(current_intent==None):
        #print(clean_input)
        #return loadIntent('params/newparams.cfg', 'Get_Tickets')
        
        if(clean_input=="flight tickets" or clean_input=="tickets" or clean_input=="Flights" or clean_input=="help me to find flight tickets" or clean_input=="flight booking" or clean_input=="book" or clean_input=="fast" or clean_input=="direct" or clean_input=="cheap" or clean_input=="flight"):
            return loadIntent('params/newparams.cfg', 'Get_Tickets')
        
        
        else:
            #return loadIntent('params/newparams.cfg',scores[-1][0])

            #print("else",scores[-1][0])
        #if not(clean_input=="flight tickets" or clean_input=="tickets" or clean_input=="Flights" or clean_input=="help me flight tickets" or clean_input=="flight booking" or clean_input=="book" or clean_input=="fast" or clean_input=="direct" or clean_input=="cheap" or clean_input=="flight"):
            print("Sorry, my knowledge is limited only to Flight Booking and Cancellation..")
            context = IntentComplete()
        #return 'action: ' + current_intent.action, context
        #             context = IntentComplete()
            session = Session()
            while True:

                inp = input('User: ')
                print ('BOT:', session.reply(inp))

    else:
        #If current intent is not none, stick with the ongoing intent
        return current_intent
# def getmatchingstring(listtosearch, inputstring):
#     r = re.compile(inputstring)
#     matchingstring = list(filter(r.match, listtosearch))
#     if len(matchingstring) > 0:
#         return matchingstring[0]

def getattributes(uinput,context,attributes):
    '''This function marks the entities in user input, and updates
    the attributes dictionary'''
    #Can use context to context specific attribute fetching
    if context.name.startswith('IntentComplete'):
        return attributes, uinput
    else:
        #Code can be optimised here, loading the same files each time suboptimal 
        files = os.listdir('./entities/')
        entities = {}
        for fil in files:
            lines = open('./entities/'+fil).readlines()
            for i, line in enumerate(lines):
                lines[i] = line[:-1]
            entities[fil[:-4]] = '|'.join(lines)

        #Extract entity and update it in attgoa
        for entity in entities:
            for i in entities[entity].split('|'):
                if i.lower() in uinput.lower():
                    attributes[entity] = i
        #for entity in entities:
                #uinput = re.sub(entities[entity],r'$'+entity,uinput,flags=re.IGNORECASE)

        ismatch = False
        uinput = uinput.lower()
        #Example of where the context is being used to do conditional branching.
#         Froms = ["Chennai","Tirupathi","Bangalore"]
#         matchingstring = getmatchingstring(Froms, uinput)
#         if matchingstring:
#             attributes['From'] = matchingstring
#             ismatch = True
#         else:
#             Tos = ["goa","Chennai","Tirupathi","Bangalore","Delhi","pune"]
#             matchingstring = getmatchingstring(Tos, uinput)
#             if matchingstring:
#                 attributes['Tos'] = matchingstring
#                 ismatch = True
#             else: 
#                 Dates = ["22/08/2019","23/08/2019","24/08/2019","25/08/2019","26/08/2019"]
#                 matchingstring = getmatchingstring(Dates, uinput)
#                 if matchingstring:
#                     attributes['Dates'] = matchingstring
#                     ismatch = True
        if ismatch:
            context.active = False     
            
        return attributes, uinput

In [4]:
class Session:
    def __init__(self, attributes=None, active_contexts=[FirstGreeting(), IntentComplete() ]):
        
        '''Initialise a default session'''
        
        #Active contexts not used yet, can use it to have multiple contexts
        self.active_contexts = active_contexts
        
        #Contexts are flags which control dialogue flow, see Contexts.py        
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def update_contexts(self):
        '''Not used yet, but is intended to maintain active contexts'''
        for context in self.active_contexts:
            if context.active:
                context.decrease_lifespan()

    def reply(self, user_input):
        '''Generate response to user input'''
        
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        #print(self.current_intent)
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)
        #print(prompt)
        #print(self.context)
        #prompt being None means all parameters satisfied, perform the intent action
        if prompt is None:
            if self.context.name!='IntentComplete':
                prompt, self.context = check_actions(self.current_intent, self.attributes, self.context)
        
        #Resets the state after the Intent is complete
        if self.context.name=='IntentComplete':
            self.attributes = {}
            self.context = FirstGreeting()
            self.current_intent = None
        
        return prompt

In [ ]:
session = Session()

print ('BOT: Hey there  i am  Flight Ticket booking Chat Bot \n \n I can help you with finding Flight tickets\n'+'             ',emoji.emojize(':airplane:'),emoji.emojize(':airplane:') ,emoji.emojize(':airplane:') ,emoji.emojize(':airplane:'))

while True:
    
    inp = input('User: ')
    print ('BOT:', session.reply(inp))

BOT: Hey there  i am  Flight Ticket booking Chat Bot 
 
 I can help you with finding Flight tickets
              ✈ ✈ ✈ ✈
User: help me to find flight tickets
BOT:  Sure Please enter Departure location(chennai/Tirupathi/Bangalore/Delhi/cochin) 
User: chennai
BOT:  Please Enter Destination location (Singapore/Mudurai/Mumbai/Hyderabad/jaipur)
User: singapore
BOT:  Please Enter Date of travel in format: DD/MM/YY available dates : 22/08/2019 ; 23/08/2019 ; 24/08/2019;25/08/2019;26/08/2019
User: 22/08/2019
BOT:  choose connection type cheap or fast?
User: cheap
BOT:  Please enter your ID min 6 digits
User: 123456
Some Flights matching for You... => 

1) ABC Airlines
2) XYZ Airlines
Please shoose the flight for booking
2

 Congratulations **** Your Flight:2) XYZ Airlines Booked succesfully. 
 Referance NO. for your booking : =>  2) 1290231,53523,4324656,78867,2426475,78675
BOT: action: Get_Tickets
User: dance show
Sorry, my knowledge is limited only to Flight Booking and Cancellation..
User: